In [0]:
%run ../utils/common

In [0]:
dbutils.widgets.text("environment", "", "")
environment = dbutils.widgets.get("environment")

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
spark.sql(f"""create or replace view {catalog_name}.udp_wcm_gold_vcm_dwh.view_d_vinid_customers_full
          as 
          WITH CITY AS (
  SELECT DISTINCT
    C.city_name,
    C.city_id
  FROM
    {catalog_name}.udp_wcm_silver_dim.cfg_city_district_ward as C
),
DIST AS (
  SELECT DISTINCT
    D.district_id,
    D.district_name
  FROM
    {catalog_name}.udp_wcm_silver_dim.cfg_city_district_ward as D
),
WARD AS (
  SELECT DISTINCT
    W.ward_id,
    W.ward_name
  FROM
    {catalog_name}.udp_wcm_silver_dim.cfg_city_district_ward as W
),
CUS_ADD AS (
  SELECT
    C.* EXCEPT (csn, full_address),
    CONCAT(
      C.full_address,
      CASE
        WHEN W.ward_name IS NOT NULL THEN CONCAT(',', W.ward_name)
        ELSE ''
      END,
      CASE
        WHEN D.district_name IS NOT NULL THEN CONCAT(',', D.district_name)
        ELSE ''
      END,
      CASE
        WHEN T.city_name IS NOT NULL THEN CONCAT(',', T.city_name)
        ELSE ''
      END
    ) AS full_address,
    CAST(csn AS STRING) AS csn,
    T.city_name,
    D.district_name,
    W.ward_name
  FROM
    {catalog_name}.udp_wcm_silver_dim.d_vinid_customers C
      LEFT JOIN CITY T
        ON T.city_id = C.city
      LEFT JOIN WARD W
        ON W.ward_id = C.ward
      LEFT JOIN DIST D
        ON D.district_id = C.district
),
SAP_CUS AS (
  SELECT
    *,
    CONCAT(
      C.full_address, " ", C.adress_2, " ", A.ward_name, " ", A.district_name, " ", A.city_name
    ) AS full_address_2
  FROM
    {catalog_name}.udp_wcm_silver_dim.d_customer C
      LEFT JOIN {catalog_name}.udp_wcm_silver_dim.cfg_city_district_ward A
        ON C.adress_3 = CAST(A.ward_id AS STRING)
)
SELECT
  SC.csn,
  SC.account_no AS vinid_card_number_8star,
  COALESCE(vc.full_name, SC.full_name) AS full_name,
  COALESCE(VC.dob, SC.dob) AS dob,
  COALESCE(VC.gender, SC.gender) AS gender,
  COALESCE(VC.mobile_number, SC.mobile_number) as mobile_number,
  REPLACE(COALESCE(vc.full_address, SC.full_address_2), '"', '') AS full_address,
  COALESCE(VC.email_address, SC.email_address) AS email_address,
  SC.adress_2 AS sap_adress_2,
  SC.adress_3 AS sap_adress_3,
  VC.cmnd_passport AS cmnd_passport,
  VC.reg_date,
  VC.city AS vid_city_id,
  VC.city_name,
  VC.district AS vid_district_id,
  VC.district_name,
  VC.ward AS vid_ward_id,
  VC.ward_name,
  VC.home_phone AS vid_home_phone,
  VC.country AS vid_country,
  VC.address2 AS vid_address2,
  VC.address3 AS vid_address3,
  VC.address4 AS vid_address4,
  CAST(months_between(current_date(), COALESCE(vc.dob, SC.dob)) / 12 AS INT) AS age
FROM
  SAP_CUS SC
    LEFT JOIN CUS_ADD as VC
      ON SC.csn = VC.csn
UNION ALL
SELECT
  VC.csn,
  NULL AS vinid_card_number_8star,
  vc.full_name AS full_name,
  VC.dob AS dob,
  VC.gender AS gender,
  VC.mobile_number as mobile_number,
  REPLACE(VC.full_address, '"', '') AS full_address,
  VC.email_address AS email_address,
  NULL AS sap_adress_2,
  NULL AS sap_adress_3,
  VC.cmnd_passport AS cmnd_passport,
  VC.reg_date,
  VC.city AS vid_city_id,
  VC.city_name,
  VC.district AS vid_district_id,
  VC.district_name,
  VC.ward AS vid_ward_id,
  VC.ward_name,
  VC.home_phone AS vid_home_phone,
  VC.country AS vid_country,
  VC.address2 AS vid_address2,
  VC.address3 AS vid_address3,
  VC.address4 AS vid_address4,
  CAST(months_between(current_date(), COALESCE(vc.dob, SC.dob)) / 12 AS INT) AS age
FROM
  CUS_ADD VC
    LEFT JOIN SAP_CUS as SC
      ON SC.csn = VC.csn
WHERE
  SC.csn IS NULL""")